<br/><br/><br/><br/><br/>

## 주석이 있는 셀이 나오기 전까지 셀들을 실행해주세요.


<br/><br/><br/><br/><br/>

In [1]:
import pandas as pd
import numpy as np
import scipy
import scipy.sparse
import scipy.stats
import os
import scipy.io as sio
%matplotlib inline
from pylab import *

import warnings
warnings.filterwarnings(action='ignore')

from keras.models import load_model
from pkg_resources import resource_filename
from utils2 import one_hot_encode

import datetime as dt
import h5py
import numpy as np
import fnmatch
import time
import re
from sklearn.metrics import average_precision_score
from pandas import DataFrame


bases = ['A','T','C','G']
dna_dict = dict(zip(list('ATCG'),range(4)))
watsoncrick = {'N':'N','.':'.','C':'G','G':'C','A':'T','T':'A','*':'*'}

def add_base(li):
		new_li = []
		for s in li:
			for b in bases:
				new_li.append(s+b)
		return new_li

def make_mer_list(mer_len):
	li = bases
	for i in range(mer_len-1):
		li = add_base(li)
	return li

def get_energy(X,w,w0):
    B = X*w+w0;
    mx = amax(B);
    Z = scipy.matrix(mx+log(scipy.sum(exp(B-mx),axis=1)));
    try:
        B = B-Z
    except:
        B = B-Z.transpose()
    return B

In [2]:
rosen_data = sio.loadmat('rosen_weight/Reads.mat')

A5SS_data = rosen_data['A5SS']
A5SS_reads = np.array(A5SS_data.sum(1)).flatten()
A5SS_data = np.array(A5SS_data.todense())

all_index_265137=np.array(range(265137))
nonzero_index = A5SS_data.sum(axis=1)!=0

using_index = all_index_265137[nonzero_index]

A5SS_reads = A5SS_reads[using_index]
A5SS_data = A5SS_data[using_index]
A5SS_data = A5SS_data/A5SS_data.sum(axis=1)[:,newaxis]
A5SS_seqs = pd.read_csv('rosen_weight/A5SS_Seqs.csv',index_col=0).Seq[using_index]

In [3]:
mlr_weight_data = sio.loadmat('rosen_weight/saving_weight.mat')
all_prediction_of_mlr = mlr_weight_data['Prediction']
w_mlr=mlr_weight_data['Mer_scores']
w0_mlr=mlr_weight_data['w0']
wfull_mlr=mlr_weight_data['wfull']

In [4]:
paths1 = ('rosen_weight/SpliceAI400_g{}.h5'.format(x) for x in [1,2,3,4,5])
models_400=[load_model(resource_filename('spliceai', x),compile=False) for x in paths1]

paths2 = ('rosen_weight/reg_a5ss.h5' for x in [1])
models_reg=[load_model(resource_filename('spliceai', x),compile=False) for x in paths2]

In [5]:
def test_with_mlr(index,all_seqs=A5SS_seqs,wfull_mlr=wfull_mlr,w0_mlr=w0_mlr):
    def seq_to_region(index,all_seqs=A5SS_seqs):
        cur_seq=all_seqs[index]

        mer_dict = dict(zip(make_mer_list(6),range(4**6)))
        rows,cols,vals = [],[],[]

        for b in (list(range(7-6,32))+list(range(50-6,75))):
            cur_mer = cur_seq[b:b+6]
            cur_mer_list_pos = mer_dict[cur_mer]
            rows.append(0)
            cols.append(cur_mer_list_pos+4**6*(b+80))
            vals.append(1)

        rows.append(0)
        cols.append(4**6*230-1)
        vals.append(0)
        X = scipy.sparse.csr_matrix((vals,(rows,cols)))
        return X
    
    a6=seq_to_region(index)
    
    return exp(get_energy(a6,wfull_mlr,w0_mlr))

In [6]:
def test_with_400_model(index,all_seqs=A5SS_seqs,models=models_400):
    a5ss_full='atggtgtccaagggcgaggagctgttcaccggggtggtgcccatcctggtcgagctggacggcgacgtaaacggccacaagttcagcgtcagcggcgagggcgagggcgatgccacctacggcaaactgaccctgaagttcatctgcaccaccggcaagctgcccgtgccctggcccaccctcgtgaccaccttcggctacggcctgatgtgcttcgcccgctaccccgaccacatgaagcagcacgacttcttcaagtccgccatgcccgaaggctacgtccaggagcgcaccatcttcttcaaggacgacggcaactacaagacccgcgccgaagtgaagttcgagggcgacaccctcgtgaaccgcatcgagctaaagggcatcgacttcaaggaggacggcaacatcctggggcacaagctggagtacaactacaacagccacaacgtctatatcatggccgacaagcagaagaacggcatcaaagtgaacttcaagatccgccacaacatcgaggtgcttggnnnnnnnnnnnnnnnnnnnnnnnnnggtcgacccaggttcgtgnnnnnnnnnnnnnnnnnnnnnnnnngaggtattcttatcaccttcgtggctacagagtttccttatttgtctctgttgccggcttatatggacaagcatatcacagccatttatcggagcgcctccgtacacgctattatcggacgcctcgcgagatcaatacgtataccagctgccctcgatacatgtcttggcatcgtttgcttctcgagtactacctggttcctcttctttctttctcttctctttcaggacggcagcgtgcagctcgccgaccactaccagcagaacacccccatcggcgacggccccgtgctgctgcccgacaaccactacctgagctaccagtccgccctgagcaaagaccccaacgagaagcgcgatcacatggtcctgctggagttcgtgaccgccgccgggatcactctcggcatggacgagctgtacaagga'.upper()
    input_sequence=a5ss_full[:520]+A5SS_seqs[index]+a5ss_full[621:]
    
    context = 400

    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    
    donor_prob = y[0, :, 2]
    
    return donor_prob

In [7]:
def test_with_reg_model(index,all_seqs=A5SS_seqs,models=models_reg):
    a5ss_full='atggtgtccaagggcgaggagctgttcaccggggtggtgcccatcctggtcgagctggacggcgacgtaaacggccacaagttcagcgtcagcggcgagggcgagggcgatgccacctacggcaaactgaccctgaagttcatctgcaccaccggcaagctgcccgtgccctggcccaccctcgtgaccaccttcggctacggcctgatgtgcttcgcccgctaccccgaccacatgaagcagcacgacttcttcaagtccgccatgcccgaaggctacgtccaggagcgcaccatcttcttcaaggacgacggcaactacaagacccgcgccgaagtgaagttcgagggcgacaccctcgtgaaccgcatcgagctaaagggcatcgacttcaaggaggacggcaacatcctggggcacaagctggagtacaactacaacagccacaacgtctatatcatggccgacaagcagaagaacggcatcaaagtgaacttcaagatccgccacaacatcgaggtgcttggnnnnnnnnnnnnnnnnnnnnnnnnnggtcgacccaggttcgtgnnnnnnnnnnnnnnnnnnnnnnnnngaggtattcttatcaccttcgtggctacagagtttccttatttgtctctgttgccggcttatatggacaagcatatcacagccatttatcggagcgcctccgtacacgctattatcggacgcctcgcgagatcaatacgtataccagctgccctcgatacatgtcttggcatcgtttgcttctcgagtactacctggttcctcttctttctttctcttctctttcaggacggcagcgtgcagctcgccgaccactaccagcagaacacccccatcggcgacggccccgtgctgctgcccgacaaccactacctgagctaccagtccgccctgagcaaagaccccaacgagaagcgcgatcacatggtcctgctggagttcgtgaccgccgccgggatcactctcggcatggacgagctgtacaagga'.upper()
    input_sequence=a5ss_full[:520]+A5SS_seqs[index]+a5ss_full[621:]
    
    context = 400

    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(1)], axis=0)
    
    donor_prob = y[0, :, 2]
    
    return donor_prob

In [27]:
def compare_three_model(index,all_seqs=A5SS_seqs,models_400=models_400,models_reg=models_reg,wfull_mlr=wfull_mlr,w0_mlr=w0_mlr):
    result_mlr = test_with_mlr(index,all_seqs=A5SS_seqs,wfull_mlr=wfull_mlr,w0_mlr=w0_mlr)
    result_400 = test_with_400_model(index,all_seqs=A5SS_seqs,models=models_400)
    result_reg = test_with_reg_model(index,all_seqs=A5SS_seqs,models=models_reg)
    
    result_mlr_crop=np.array(result_mlr)[0][:-1]
    result_400_crop=result_400[518:518+80]
    result_reg_crop=result_reg[518:518+80]
    
    ground_truth_label = A5SS_data[index]
    target_no_splicing = float(A5SS_data[index][-1])
    
    gt_list = []
    for i in range(80):
        if A5SS_data[index][i]>0.001:
            gt_list.append([A5SS_data[index][i],i])
    
    gt_list.sort()

    list_for_index = []
    list_for_prob = []
    
    
    pred_of_mlr=[]
    pred_of_400=[]
    pred_of_reg=[]
    
    remain_400 = 1.0
    remain_reg = 1.0
    
    
    for i in range(len(gt_list)-1,-1,-1):
        list_for_index.append(gt_list[i][1])
        list_for_prob.append(gt_list[i][0])
        
        remain_400-=result_400_crop[gt_list[i][1]]
        remain_reg-=result_reg_crop[gt_list[i][1]]
        
        pred_of_mlr.append(result_mlr_crop[gt_list[i][1]])
        pred_of_400.append(result_400_crop[gt_list[i][1]])
        pred_of_reg.append(result_reg_crop[gt_list[i][1]])

        
    pred_of_mlr.append(float(np.array(result_mlr)[0][-1]))
    pred_of_400.append(remain_400)
    pred_of_reg.append(remain_reg)
        
    list_for_index.append('no splicing')
    list_for_prob.append(target_no_splicing)
    
    

    table_for_comparison=pd.DataFrame()
    
    table_for_comparison['GT index'] = list_for_index
    table_for_comparison['GT prob'] = list_for_prob
        
    table_for_comparison['MLR'] = pred_of_mlr
    table_for_comparison['400'] = pred_of_400
    table_for_comparison['Reg'] = pred_of_reg
    
    loss_of_mlr = (target_no_splicing-pred_of_mlr[-1])**2
    loss_of_400 = (target_no_splicing-pred_of_400[-1])**2
    loss_of_reg = (target_no_splicing-pred_of_reg[-1])**2
    
    
    
        
    
    
    for i in range(80):
        loss_of_mlr += (result_mlr_crop[i]-ground_truth_label[i])**2
        loss_of_400 += (result_400_crop[i]-ground_truth_label[i])**2
        loss_of_reg += (result_reg_crop[i]-ground_truth_label[i])**2
    
    print('GG'+A5SS_seqs[index])
    
    for i in range(80):
        if i in list_for_index:
            print('^',end="")
        else:
            print(' ',end="")
    
    print()
    print()
    print("Loss of MLR : "+str(loss_of_mlr))
    print("Loss of 400 : "+str(loss_of_400))
    print("Loss of Reg : "+str(loss_of_reg))
    
    
    return table_for_comparison

<br/><br/><br/>

MLR 모델의 입력이 상당히 제약되어있어서 우선은 MLR을 학습하는데에 사용한 데이터들에 대해서만 다른 모델들과 성능을 비교해볼 수 있는 코드를 작성하였습니다. 비교 모델은 두 개로

1. context가 400인 spliceai 모델
2. 레이블을 변경한([0.8, 0.0, 0.2] 형식) 데이터로 학습한 context 400 spliceai 모델

compare_three_model의 인자로 0부터 265,043의 input을 줄 수 있고 함수를 실행하면 제일 먼저 입력된 sequence가 출력됩니다.

눈웃음 문자(^)를 통해 실제(Groudn Truth) Donor가 어디인지 확인하기 쉽게 하였습니다.

<br/><br/>

다음 세 줄에는 세 모델의 Loss가 나타납니다. (낮을수록 높은 정확도)

MLR 모델이 80개의 site에 대해서만 Prob를 출력하였기에 해당 80개의 index에 대해서만 다른 모델들이 예측한 Donor probability와 비교를 진행합니다. 모든 seq에 대한 각 모델들의 Loss들의 합은 오늘 15시 전까지 최대한 계산을 진행할 예정입니다.

<br/><br/>
마지막으로 표에서는 GT Donor index와 해당 위치에서 스플라이싱이 일어난 확률 GT prob와 더불어 각 모델들의 해당 위치에 대한 Prob prediction을 확인할 수 있습니다. GT prob에 가까울수록 학습이 잘 된 것일겁니다.

In [28]:
compare_three_model(5)

GGTGCTTGGATTCATAGCGAAGAGAGAAGAGCGAGGTCGACCCAGGTTCGTGCGATGGCCCGTTGCGCTGGTACTGGGAGGTATTCTTATCACCTTCGTGGCT
                                            ^                        ^          

Loss of MLR : 0.3010723940053903
Loss of 400 : 0.2576762296309788
Loss of Reg : 0.278469889569471


,GT index,GT prob,MLR,400,Reg
0,44,0.560606,0.954584,0.893872,0.928382
1,69,0.378788,0.001071,0.006462,0.001301
2,no splicing,0.060606,0.011954,0.099666,0.070317


In [32]:
def compare_three_model_loss(index,all_seqs=A5SS_seqs,models_400=models_400,models_reg=models_reg,wfull_mlr=wfull_mlr,w0_mlr=w0_mlr):
    result_mlr = test_with_mlr(index,all_seqs=A5SS_seqs,wfull_mlr=wfull_mlr,w0_mlr=w0_mlr)
    result_400 = test_with_400_model(index,all_seqs=A5SS_seqs,models=models_400)
    result_reg = test_with_reg_model(index,all_seqs=A5SS_seqs,models=models_reg)
    
    result_mlr_crop=np.array(result_mlr)[0][:-1]
    result_400_crop=result_400[518:518+80]
    result_reg_crop=result_reg[518:518+80]
    
    ground_truth_label = A5SS_data[index]
    target_no_splicing = float(A5SS_data[index][-1])
    
    gt_list = []
    for i in range(80):
        if A5SS_data[index][i]>0.001:
            gt_list.append([A5SS_data[index][i],i])
    
    gt_list.sort()

    list_for_index = []
    list_for_prob = []
    
    
    pred_of_mlr=[]
    pred_of_400=[]
    pred_of_reg=[]
    
    remain_400 = 1.0
    remain_reg = 1.0
    
    
    for i in range(len(gt_list)-1,-1,-1):
        list_for_index.append(gt_list[i][1])
        list_for_prob.append(gt_list[i][0])
        
        remain_400-=result_400_crop[gt_list[i][1]]
        remain_reg-=result_reg_crop[gt_list[i][1]]
        
        pred_of_mlr.append(result_mlr_crop[gt_list[i][1]])
        pred_of_400.append(result_400_crop[gt_list[i][1]])
        pred_of_reg.append(result_reg_crop[gt_list[i][1]])

        
    pred_of_mlr.append(float(np.array(result_mlr)[0][-1]))
    pred_of_400.append(remain_400)
    pred_of_reg.append(remain_reg)
        
    list_for_index.append('no splicing')
    list_for_prob.append(target_no_splicing)
    
    

    table_for_comparison=pd.DataFrame()
    
    table_for_comparison['GT index'] = list_for_index
    table_for_comparison['GT prob'] = list_for_prob
        
    table_for_comparison['MLR'] = pred_of_mlr
    table_for_comparison['400'] = pred_of_400
    table_for_comparison['Reg'] = pred_of_reg
    
    loss_of_mlr = (target_no_splicing-pred_of_mlr[-1])**2
    loss_of_400 = (target_no_splicing-pred_of_400[-1])**2
    loss_of_reg = (target_no_splicing-pred_of_reg[-1])**2
    
    for i in range(80):
        loss_of_mlr += (result_mlr_crop[i]-ground_truth_label[i])**2
        loss_of_400 += (result_400_crop[i]-ground_truth_label[i])**2
        loss_of_reg += (result_reg_crop[i]-ground_truth_label[i])**2
    
    return loss_of_mlr,loss_of_400,loss_of_reg

In [33]:
loss_mlr=0.0
loss_400=0.0
loss_reg=0.0

In [34]:
for i in range(265044):
    losses=compare_three_model_loss(i)
    loss_mlr += losses[0]
    loss_400 += losses[1]
    loss_reg += losses[2]
    
    if (i+1)%200==0:
        print(str(i+1)+' : '+str(loss_mlr)+', '+str(loss_400)+', '+str(loss_reg))


200 : 14.034362739025031, 53.870041360677035, 10.327790633518767
400 : 31.27900068271353, 110.948312118355, 23.351025289971734
600 : 47.69568887284781, 165.29272975364836, 34.14324705077355
800 : 63.45252073881033, 221.36201482632674, 44.962304772366394
1000 : 79.41643205717467, 266.61665434277546, 57.205967268236215
1200 : 95.40796434105665, 317.07767085236003, 71.04337881361691
1400 : 113.04432143028748, 371.2019594311144, 83.50086954826428
1600 : 128.12318720366795, 423.12193716752626, 94.83987944604561


KeyboardInterrupt: 

In [ ]:
scvsvvv